In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import pandas as pd


In [3]:
pd.set_option('display.max_columns', None)
df_resort=pd.read_pickle("cleaned_resort_bookings.pkl")
df_resort.head(15)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,arrival_date,country_grouped,lead_time_scaled,required_car_parking_spaces_scaled,special_requests_clipped,special_requests_scaled,deposit_paid,got_requested_room_type
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01,2015-07-01,PRT,2.280972,-0.256841,0,-0.728703,0,1
1,Resort Hotel,0,444,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01,2015-07-01,PRT,3.256111,-0.256841,0,-0.728703,0,1
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02,2015-07-01,GBR,-0.921690,-0.256841,0,-0.728703,0,0
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02,2015-07-01,GBR,-0.864328,-0.256841,0,-0.728703,0,1
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03,2015-07-01,GBR,-0.854768,-0.256841,1,0.554576,0,1
5,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03,2015-07-01,GBR,-0.854768,-0.256841,1,0.554576,0,1
6,Resort Hotel,0,0,2015,July,27,1,0,2,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,0,No Deposit,NaN,NaN,0,Transient,107.00,0,0,Check-Out,2015-07-03,2015-07-01,PRT,-0.988611,-0.256841,0,-0.728703,0,1
7,Resort Hotel,0,9,2015,July,27,1,0,2,2,0.0,0,FB,PRT,Direct,Direct,0,0,0,C,C,0,No Deposit,303.0,NaN,0,Transient,103.00,0,1,Check-Out,2015-07-03,2015-07-01,PRT,-0.902569,-0.256841,1,0.554576,0,1
8,Resort Hotel,1,85,2015,July,27,1,0,3,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,82.00,0,1,Canceled,2015-05-06,2015-07-01,PRT,-0.175995,-0.256841,1,0.554576,0,1
9,Resort Hotel,1,75,2015,July,27,1,0,3,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,D,D,0,No Deposit,15.0,NaN,0,Transient,105.50,0,0,Canceled,2015-04-22,2015-07-01,PRT,-0.271597,-0.256841,0,-0.728703,0,1


In [5]:
features = [
    'lead_time_scaled',
    'required_car_parking_spaces_scaled',
    'special_requests_scaled',
    'market_segment',
    'distribution_channel',
    'deposit_paid',
    'got_requested_room_type',
    'customer_type',
    'country_grouped'
    ]
target = 'is_canceled'
X = df_resort[features]
y = df_resort[target]


In [6]:
X_encoded = pd.get_dummies(X, columns=['market_segment','distribution_channel','deposit_paid','got_requested_room_type','customer_type','country_grouped'], drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

In [7]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)[:,1]
print("AUC:", roc_auc_score(y_test, y_pred_proba))

AUC: 0.8903394624087773


In [8]:
df_test = X_test.copy()
df_test['storno_score'] = y_pred_proba

In [9]:
df_test.head(20)

,lead_time_scaled,required_car_parking_spaces_scaled,special_requests_scaled,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,distribution_channel_Direct,distribution_channel_TA/TO,deposit_paid_1,got_requested_room_type_1,customer_type_Transient,customer_type_Transient-Party,country_grouped_BRA,country_grouped_DEU,country_grouped_ESP,country_grouped_FRA,country_grouped_GBR,country_grouped_IRL,country_grouped_ITA,country_grouped_NLD,country_grouped_Other,country_grouped_PRT,storno_score
21649,-0.740046,-0.256841,-0.728703,False,True,False,False,False,True,False,False,True,False,True,False,False,False,False,False,False,False,False,False,True,0.165011
1806,-0.988611,-0.256841,0.554576,False,False,False,False,True,False,True,False,True,True,False,False,False,False,False,False,False,False,True,False,False,0.143610
22001,-0.663565,-0.256841,0.554576,False,False,False,True,False,False,True,False,True,True,False,False,False,False,False,True,False,False,False,False,False,0.023353
26758,0.550579,-0.256841,1.837855,False,False,False,True,False,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,True,0.282929
28923,1.057269,-0.256841,-0.728703,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,0.224571
39681,-0.539282,-0.256841,0.554576,False,True,False,False,False,True,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,0.088438
29722,1.802963,-0.256841,0.554576,False,False,True,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,True,False,0.269861
5751,1.267593,-0.256841,-0.728703,False,False,True,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,True,0.729689
36006,-0.042153,-0.256841,0.554576,False,False,False,True,False,False,True,False,True,True,False,False,False,False,True,False,False,False,False,False,False,0.063690
35596,-0.768727,-0.256841,0.554576,False,False,True,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,True,False,0.049510
